In [22]:
# import libraries and packages for the project
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from bs4 import BeautifulSoup
import urllib.request 
from docx import Document
from urllib.request import urlopen
import cv2
import numpy as np

In [23]:
# options of webdriver
options = webdriver.ChromeOptions()
options.add_argument('--disable-notifications')
options.add_argument('--incognito')
# open the browser
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
url = 'https://novelsonline.net/tensei-shitara-slime-datta-ken-ln/volume-1/chapter-1-1'
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\TTC\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\TTC\AppData\Local\Temp/ipykernel_25464/88181232.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [ ]:

page_source = BeautifulSoup(driver.page_source)
urls = page_source.find("select",{"onchange":"location = this.options[this.selectedIndex].value;"}).find_all("option")
for url in urls:
    print(url.get("value"))
    # opening the url for reading
    try:
        html = urllib.request.urlopen(url.get("value"))
    except:
        print("error")
        continue
    
    # parsing the html file
    htmlParse = BeautifulSoup(html, 'html.parser')
    
    # file name document
    filename = url.get("value").replace("https://novelsonline.net/tensei-shitara-slime-datta-ken-ln","").replace("/","_").replace("-","_") + '.docx'
    document = Document()
    # getting the value of body
    table = htmlParse.find("div", {"id": "contentall"})
    for para in table.find_all("p"):
        # write the paragraph to the document
        if(para.string != None):
            document.add_paragraph(para.string)
        # write the image to the document
        if(para.find("img") != None):
            # getting the image url
            req = urlopen("https://novelsonline.net" + para.find("img").get("src"))
            # getting the image data
            arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
            # decode the image
            img = cv2.imdecode(arr, -1)
            # name the image
            filenameimg = para.find("img").get("src").replace("/","_").replace("-","_").replace(".","_") + '.jpg' #Có thể viết lại tên cho đỡ ngáo
            # save the image
            cv2.imwrite(filenameimg, img)
            # add the image to the document
            document.add_picture(filenameimg)
    # save the document
    document.save(filename)

